In [1]:
import SimpleITK as sitk
import numpy as np
from radiomics import featureextractor, getTestCase
import itertools
import h5py
import shutil
import pandas as pd
import tempfile
import os
from functools import partial
import multiprocessing as mp
# import radiomics

In [9]:
path_deauvill_df="/workspaces/pilot_lymphoma/data/all_deauville_form.csv"
path_folder_files="/root/data/prepared_registered"
deauvill_df=pd.read_csv(path_deauvill_df)

curr_row=list(deauvill_df.iterrows())[0][1]

#extract necessary information from dataframe
pat_id= curr_row["pat_id"]
deauville= curr_row["deauville_1"]
study_0_or_1= curr_row["study_0_or_1"]

#get paths to the files from main folder
reg_form="lin_transf"
path_curr_folder=f"{path_folder_files}/pat_{pat_id}/{reg_form}"
path_SUV= f"{path_curr_folder}/study_{study_0_or_1}_SUVS.nii.gz"
path_CT= f"{path_curr_folder}/study_{study_0_or_1}_ct_soft.nii.gz"

#load the images
img_SUV=sitk.ReadImage(path_SUV)
img_CT=sitk.ReadImage(path_CT)





pat_id            2
deauville_1     1/2
study_0_or_1      0
Name: 0, dtype: object

In [11]:
main_loc_path="/root/data/prepared_registered/pat_2"

suv_0_p=f"{main_loc_path}/study_0_SUVS.nii.gz"
suv_0=sitk.ReadImage(suv_0_p)

suv_1_p=f"{main_loc_path}/study_1_SUVS.nii.gz"
suv_1=sitk.ReadImage(suv_1_p)

# lintransf_p=f"{main_loc_path}/lin_transf.tfm"
# lintransf=sitk.ReadTransform(lintransf_p)
# suv_0=sitk.Resample(suv_0, lintransf, sitk.sitkBSpline, 0.0, suv_0.GetPixelID())
# sitk.WriteImage(suv_0, "/workspaces/pilot_lymphoma/data/fully_preprocessed/pat_2/study_0_SUVS_registered.nii.gz")


In [24]:
import SimpleITK as sitk

# Define the fixed and moving images
fixed_image = suv_1 #sitk.ReadImage('fixed_image.nii.gz', sitk.sitkFloat32)
moving_image = suv_0#sitk.ReadImage('moving_image.nii.gz', sitk.sitkFloat32)

# Define the fiducial points in the fixed and moving images
fixed_points = np.load(f"{main_loc_path}/To.npy")#[[10.0, 10.0, 10.0],[9.0, 9.0, 9.0]]
shh=fixed_points.shape
fixed_points = [c for p in fixed_points for c in p]

print(fixed_points)
 #[sitk.VectorDouble([10, 10, 10]), sitk.VectorDouble([20, 20, 20]), sitk.VectorDouble([30, 30, 30])]
moving_points = np.load(f"{main_loc_path}/From.npy")#[[10.0, 10.0, 10.0],[9.0, 9.0, 9.0]]
moving_points =[c for p in moving_points for c in p] #[sitk.VectorDouble([15, 15, 15]), sitk.VectorDouble([25, 25, 25]), sitk.VectorDouble([35, 35, 35])]

# Create the transformation
initial_transform = sitk.LandmarkBasedTransformInitializer(sitk.VersorRigid3DTransform(), fixed_points, moving_points,numberOfControlPoints=shh[0])

# Perform the registration
registered_image = sitk.Resample(moving_image, fixed_image, initial_transform, sitk.sitkBSpline, 0.0, moving_image.GetPixelID())

# Write the registered image to a file
sitk.WriteImage(registered_image, "/workspaces/pilot_lymphoma/data/fully_preprocessed/pat_2/study_0_SUVS_registered.nii.gz")

(6, 3)
[-137.23987719038843, 104.35994720458984, -1007.3944981534718, 62.3861664248185, 132.86975860595703, -1603.9780767736302, -66.10875820106179, 145.08824920654297, -1606.2726289990924, -0.7140197753905682, 226.54485321044925, -1111.7966244119991, 15.577301025390682, 251.7849276905328, -1221.9351312341826, 25.90278603997035, 199.5838645612689, -896.5000000000002]
